# HMM GMM

We use 3-state mono-phone HMMs to construct this recognizer. The emission probability of every state is modeled by a GMM. Say we have F mono-phones (F = 39 in our English lexicon), and a G-mixture GMM for each mono-phone state. Thus the GMM-HMM has 3FG mixture components in total. Compared to GMM-UBM, these mixtures are better separated in the phoneme space.

MFCC:  MFCCs  are  extracted  from  16kHz  utterance  with  40 filter-banks distributed between 0 and 8kHz. Static 19-dimensional coefficients plus energy and their delta and delta-delta form a 60-dimensional vector. CMVN is applied per utterance.

Hmm, deepspeech ma 26 MFCC features i zdaje się, że usuwa co drugi, ale to w związku z jakąś sztuczką uczenia RNN. Mają tak: features = mfcc(audio, samplerate=fs, numcep=numcep) gdzie mfcc jest z psf jak u nas xD a numcep=26 domyślnie, a fs=16000 domyślnie

Hmm, inny papier: In our HMM-based method, a phoneme recognizer is firsttrained with 3-state, GMM-based, mono-phone HMMs. Thisrecognizer is the same as in speech recognition. LetFbe thetotal number of mono-phones (i.e. 39),S=3Fbe the numberof all states,Gthe number of Gaussian components per state,andC=SGthe number of all individual Gaussians, and let(s, g)denote Gaussian componentgin states.
3-state - what are you?

Given a transcription, a graph of HMM is composed.

The  Viterbi  and  forward-backward  (FB)  algorithms  are  twomeans to align frames to states and mixtures.

Speaker adaption is the same with Eq. 4 except mixtures here are phonetic dependent.

During the test phase,  the Viterbi-based log-likelihood ratio is expressed as:
sum_t log P(x_t | model_user,qt) - log P(x_t | model_ubm,qt)

HMM: To generate the alignment for the HMM-based modeling,we use MFCCs to train the HMM. 39 mono-phones plus a silencemodel are used,  each of which contains 3 states.  To model thecomplexity of silence, a GMM with 16 mixtures is used for every silence state, while other states are all modeled by 8 Gaussians,resulting 984 Gaussians in total. This HMM is further extended toa triphone system and remains 2142 senones.  The transcriptionsfor  DNN  training  is  generated  by  the  senone  alignment.   OnlyMFCCs are used for HMM training and alignment.

GMM-HMM and i-vector/HMM: The GMM of every state is re-estimated using the HMM alignments and different speaker fea-tures.   The total number of mixtures in our model is 984.   Thedimension of i-vector is again set to 600.  Viterbi and FB align-ments are both investigated

* 39 x 3 states

## MAP - Książka Rabinera:

MAP adaptation of speaker-independent model to a speaker of parameters is obtained by solving 

$$\frac{d}{d\lambda} P(\lambda | O) = 0$$

where

$$P(\lambda | O) = \frac{P(O | \lambda)P(\lambda)}{P(O)}$$

If we have a good prior estimated from a lot of data, then we can use it to get a model for a speaker for which data is scarce.

Conjugate priors for a random vector is defined as the prior distribution for the parameters of the parameters of the probaiblity density function of the random vector, such that the posterior distribution $P(\lambda | O)$ and the prior distribution $P(\lambda)$ belong to the same distribution family for any sample observations $O$.
For example, it is well known that the conjugate prior for the mean of a Gaussian density is also a Gaussian density.

$$\bar{u_{MAP}} = \frac{n \tau^2}{\sigma^2 + n \tau^2} \bar{o} + \frac{\sigma^2}{\sigma^2 + n \tau^2} \rho$$

where $\bar{o}$ - sample mean of new data, $n$ - number of training observations, $\tau^2$ - variance of prior, $\rho$ - mean of prior distribution

How do we estimate $\rho$ and $\tau^2$ of prior? From a collection of speaker-dependent models (and $c_m$ is a weight of a model) or from a speaker-independent model with mixture of distributions in each state ($c_m$ is a weight of a mixture component)

$\rho = \sum_{m=1}^M c_m \rho_m$

$\tau^2 = \sum_{m=1}^M c_m (\rho_m - \rho)$

System do rozpoznawania słowa:

po jednym modelu na słowo, rozpoznając matchujemy z każdym po kolei i bierzemy max prob
model left-to-right zamiast ergodic :OOOO

https://web.stanford.edu/class/cs224s/lectures/

Hmm, Viterbi learning, hard assignment: Bahm ~ o3 
EM learning, soft assignment: Bahm ~ 0*o1 + 0.15*o2 + 0.5*o3 + 0.05*o

Typical training procedure in LVCSR
Generate a forced alignment with existing model
Viterbi decoding with a very constrained prior (the transcript)
Assigns observations to HMM states
Create new observation models from update alignments
Iteratively repeat the above steps, occasionally introducing a more complex observation model or adding more difficult training examples

Viterbi Beam search

Applying FB to speech: Caveats
Network structure of HMM is always created by hand
no algorithm for double-induction of optimal structure and probabilities has been able to beat simple hand-built structures.
Always Bakisnetwork = links go forward in time
Subcase of Bakisnet: beads-on-string net

Assume that the features in the feature vector are uncorrelated
This isn’t true for FFT features, but is true for MFCC features, as we will see later


Initialization: “Flat start”Transition probabilities:set to zero any that you want to be “structurally zero”Set the rest to identical valuesLikelihoods:initialize μand sof each state to global mean and variance of all training data

Embedded TrainingGiven: phoneset, lexicon, transcribed wavefilesBuild a whole sentence HMM for each sentenceInitialize A probsto 0.5, or to zeroInitialize B probsto global mean and varianceRun multiple iterations of Baum WelchDuring each iteration, we compute forward and backward probabilitiesUse them to re-estimate A and BRun Baum-Welch until convergence

Viterbi training rather than Baum-Welch training. Computing the “Viterbi path” over the training data is called “forced alignment”
Because we know which word string to assign to each observation sequence.
We just don’t know the state sequence.
So we use aijto constrain the path to go through the correct words

Modeling phonetic context
The strongest factor affecting phonetic variability is the neighboring phone
Idea: have phone models which are specific to context
Instead of Context-Independent (CI) phonesWe’ll have Context-Dependent (CD) phones


Speech signal is not constantslope of formants, change from stop burst to releaseSo in addition to the cepstralfeaturesNeed to model changes in the cepstralfeatures over time.“delta features”“double delta” (acceleration) features

Typical MFCC featuresWindow size: 25msWindow shift: 10msPre-emphasis coefficient: 0.97MFCC:12 MFCC (mel frequency cepstral coefficients)1 energy feature12 delta MFCC features 12 double-delta MFCC features1 delta energy feature1 double-delta energy featureTotal 39-dimensional features


Acoustic Model AdaptationShift the means and variances of Gaussians to better match the input feature distributionMaximum Likelihood Linear Regression (MLLR)Maximum A Posteriori (MAP) AdaptationFor both speaker adaptation and environment adaptation

MAP Adaptation: u_new = N / (N + a) u_estimated + a / (N + a) u_old, where a is some weight of old

DUŻO BYŁO W lec5

SIL is a phoneme to a recognizerAlways inserted at start and end of utteranceCorrupting silence with bad forced alignments can break recognizer training (silence eats everything)

## Hmm, no dobra, plan działania:

Preprocessing:
wytnij ciszę z dźwięków, voice activity detection 
Wylicz MFCC, 20 stanów + d + dd 

Training:
Dla każdego sentence_id (jest ich 10):
    Utwórz HMMGMM z całego zbioru treningowego sequence_id:
        HMM with 3 states and 8 Gaussian components for each of 39 mono-phones were used for the alignment (resulting in to-tal number of 936 Gaussian components) ?? 
        albo unique(znaki) * 3 stany
        albo znaki * 3 stany trójkątna! (Bakis)
        
Enrollment:
Dla każdego gspeaker_id:
    Dla każdego sentence_id:
        Weź dane (gspeaker_id, sequence_id)
        Naucz HMM taki jak wyżej tylko na nich
        Dokonaj MAP adaptacji
        
Trial:
likelihood HMM_{podany speaker, podane zdanie} - likelihood HMM_{UBM, podane zdanie} ? 

In [1]:
%load_ext autoreload

In [168]:
%autoreload 2

import concurrent.futures as cf
import functools as ft
import itertools as it
import json
import math
import operator as op
import os

from IPython.display import display
from ipywidgets import interact, interact_manual, widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pomegranate as pg
from scipy import interpolate, linalg, misc, optimize, spatial, stats
from sklearn import metrics, mixture, cluster, utils

from paprotka.dataset import reddots
from paprotka.feature import cepstral

In [3]:
%autoreload 0

In [4]:
import warnings
warnings.filterwarnings('once')

## Load

In [5]:
root = reddots.get_root()
load_pcm = ft.partial(reddots.load_pcm, root)
load_mfcc = ft.partial(reddots.load_npy, root, 'wac2_mfcc13_ddd')

In [6]:
enrollments_1 = reddots.load_enrollments(root + '/ndx/f_part_01.trn', root + '/ndx/m_part_01.trn')
print('Enrollments', enrollments_1.dtypes, sep='\n')

trials_1 = reddots.load_trials(root + '/ndx/f_part_01.ndx', root + '/ndx/m_part_01.ndx')
print('Trials', trials_1.dtypes, sep='\n')

Enrollments
is_male                  bool
pcm_path               object
sentence_id             int16
speaker_id              int16
timestamp      datetime64[ns]
dtype: object
Trials
correct_sentence                  bool
expected_is_male                  bool
expected_sentence_id             int16
expected_speaker_id              int16
pcm_path                        object
target_person                     bool
trial_is_male                     bool
trial_sentence_id                int16
trial_speaker_id                 int16
trial_timestamp         datetime64[ns]
dtype: object


In [7]:
script = pd.read_csv(root + '/infos/phones.csv', sep=';')
print(script.loc[script['sentence_id'] == 31])

enrollments_1 = enrollments_1.merge(script, how='left', on='sentence_id')
trials_1 = trials_1.merge(script, how='left', left_on='expected_sentence_id', right_on='sentence_id')
del trials_1['sentence_id']
trials_1.rename(columns={'content': 'expected_content', 'phones': 'expected_phones'}, inplace=True)
trials_1 = trials_1.merge(script, how='left', left_on='trial_sentence_id', right_on='sentence_id')
del trials_1['sentence_id']
trials_1.rename(columns={'content': 'trial_content', 'phones': 'trial_phones'}, inplace=True)

      sentence_id                  content  \
1609           31  My voice is my password   

                                                 phones  
1609  M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D  


In [186]:
enrollments_4_td = reddots.load_enrollments(
    root + '/ndx/f_part_04_td.trn', root + '/ndx/m_part_04_td.trn'
)
enrollments_4_td = enrollments_4_td.merge(script, how='left', on='sentence_id')

In [ ]:
# enrollments_1.mfcc = enrollments_1.apply(lambda row: load_mfcc(row.pcm_path), axis=1)
# trials_1.mfcc = trials_1.apply(lambda row: load_mfcc(row.pcm_path), axis=1)
# enrollments_4_td.mfcc = enrollments_4_td.apply(lambda row: load_mfcc(row.pcm_path), axis=1)

In [164]:
# got_this_back = pg.GeneralMixtureModel(bg_model.distributions, np.exp(bg_model.weights) * bg_model.weights.sum())
# got_this_back = pg.MultivariateGaussianDistribution(means=bg_model.distributions[0].mu, covariance=bg_model.distributions[0].cov)

def with_opened_file(mode='r'):
    def decorator(fun):
        @ft.wraps(fun)
        def wrapped(path, *args, **kwargs):
            with open(path, mode=mode) as opened:
                return fun(opened, *args, **kwargs)
        return wrapped
    return decorator

def write_model(path, model):
    with open(path, 'w') as opened:
        opened.write(model.to_json())
        
def read_mvgaussian(path):
    with open(path, 'r') as opened:
        parsed = json.load(opened)
        means, covariance = parsed['parameters']
        return pg.MultivariateGaussianDistribution(means=means, covariance=covariance)
        
def read_gmm(path):
    with open(path, 'r') as opened:
        parsed = json.load(opened)
        distributions_parsed = parsed['distributions']
        weights_parsed = np.array(parsed['weights'])
        
        distributions = []
        for i, distribution in enumerate(distributions_parsed):
            try:
                distribution = pg.MultivariateGaussianDistribution(*distribution['parameters'])
                distributions.append(distribution)
            except Exception as exception:
                raise IOError('Cannot read mixture at {}'.format(i)) from exception
                        
        weights_exp = np.exp(weights_parsed) * weights_parsed.sum()
        
        return pg.GeneralMixtureModel(distributions=distributions, weights=weights_exp)
    
def write_triples(path, triples):
    for phone, triple in triples.items():
        for i, model in enumerate(triple):
            c = chr(ord('a') + i)
            write_model(path + '/{}-{}'.format(phone, c), model)
            
def read_triples(path, phones):
    triples = {}
    for phone in phones:
        triple = []
        for c in ('a', 'b', 'c'):
            try:
                model = read_gmm(path + '/{}-{}'.format(phone, c))
                triple.append(model)
            except Exception as exception:
                raise IOError('Cannot read model {}-{}'.format(phone, c)) from exception
        triples[phone] = tuple(triple)
    return triples

## Train

In [9]:
example = enrollments_1[enrollments_1.sentence_id == 31]
example_features = [load_mfcc(path) for path in example['pcm_path']]
example_lengths = [mfcc.shape[0] for mfcc in example_features] 
example_features_stack = np.vstack(example_features)

In [202]:
def wrap_with_silence(phones):
    return ['_'] + phones + ['_']

def remove_stress_combinations(phones):
    def remove_single_phone(phone):
        if phone[-1].isdigit():
            return phone[:-1]
        else:
            return phone
        
    return [remove_single_phone(phone) for phone in phones]

example_phones = example.phones.loc[0].split(' ')
example_phones_cleaned = wrap_with_silence(remove_stress_combinations(example_phones))
# example_phones_cleaned = remove_stress_combinations(example_phones)
example_phones_unique = np.unique(example_phones_cleaned)

print(example_phones)
print(example_phones_cleaned)
print(example_phones_unique)

['M', 'AY1', '_', 'V', 'OY1', 'S', '_', 'IH1', 'Z', '_', 'M', 'AY1', '_', 'P', 'AE1', 'S', 'W', 'ER2', 'D']
['_', 'M', 'AY', '_', 'V', 'OY', 'S', '_', 'IH', 'Z', '_', 'M', 'AY', '_', 'P', 'AE', 'S', 'W', 'ER', 'D', '_']
['AE' 'AY' 'D' 'ER' 'IH' 'M' 'OY' 'P' 'S' 'V' 'W' 'Z' '_']


In [12]:
bg_model = pg.GeneralMixtureModel.from_samples(
    pg.MultivariateGaussianDistribution, n_components=8, X=example_features_stack,
    verbose=True
)
bg_model.freeze()

[1] Improvement: 67269.43064035429	Time (s): 0.3719
[2] Improvement: 57030.51134584099	Time (s): 0.3453
[3] Improvement: 30093.395479215775	Time (s): 0.3203
[4] Improvement: 18284.311637017876	Time (s): 0.3124
[5] Improvement: 14776.855103159323	Time (s): 0.316
[6] Improvement: 8194.793300561607	Time (s): 0.3195
[7] Improvement: 4833.042162424419	Time (s): 0.3183
[8] Improvement: 3517.174147175625	Time (s): 0.3223
[9] Improvement: 2606.161533883307	Time (s): 0.3275
[10] Improvement: 1549.9553910815157	Time (s): 0.3235
[11] Improvement: 985.587243154645	Time (s): 0.3244
[12] Improvement: 688.077562822029	Time (s): 0.3339
[13] Improvement: 504.345995655749	Time (s): 0.3266
[14] Improvement: 421.17580865090713	Time (s): 0.3278
[15] Improvement: 397.8030948750675	Time (s): 0.3257
[16] Improvement: 357.55241839634255	Time (s): 0.325
[17] Improvement: 309.54890816425905	Time (s): 0.3248
[18] Improvement: 257.34815956139937	Time (s): 0.322
[19] Improvement: 255.9211985510774	Time (s): 0.3306


[155] Improvement: 4.96634772606194	Time (s): 0.3103
[156] Improvement: 5.728654948528856	Time (s): 0.2961
[157] Improvement: 6.747379774227738	Time (s): 0.3052
[158] Improvement: 7.176577621139586	Time (s): 0.3
[159] Improvement: 8.363533925265074	Time (s): 0.293
[160] Improvement: 9.361007482279092	Time (s): 0.2926
[161] Improvement: 11.081499905791134	Time (s): 0.2922
[162] Improvement: 14.817644377239048	Time (s): 0.2929
[163] Improvement: 20.299099640455097	Time (s): 0.2936
[164] Improvement: 25.774534576106817	Time (s): 0.2954
[165] Improvement: 30.720929235685617	Time (s): 0.2938
[166] Improvement: 34.77248046267778	Time (s): 0.2933
[167] Improvement: 36.869185243267566	Time (s): 0.2989
[168] Improvement: 37.137477449607104	Time (s): 0.3001
[169] Improvement: 34.59153809538111	Time (s): 0.2934
[170] Improvement: 34.41739834519103	Time (s): 0.2937
[171] Improvement: 34.0889338767156	Time (s): 0.2935
[172] Improvement: 30.125846203416586	Time (s): 0.2941
[173] Improvement: 28.1676

In [20]:
silence_model = pg.GeneralMixtureModel.from_samples(
    pg.MultivariateGaussianDistribution, n_components=16, X=example_features_stack,
    verbose=True
)
silence_model.freeze()

[1] Improvement: 92506.92585839983	Time (s): 0.5863
[2] Improvement: 47730.4606045573	Time (s): 0.5591
[3] Improvement: 19794.427015147172	Time (s): 0.5882
[4] Improvement: 9192.858351976145	Time (s): 0.6028
[5] Improvement: 5295.7776417764835	Time (s): 0.5956
[6] Improvement: 3588.7954395171255	Time (s): 0.6035
[7] Improvement: 2925.9529533958994	Time (s): 0.6143
[8] Improvement: 2345.782454967033	Time (s): 0.6105
[9] Improvement: 1843.6297330958769	Time (s): 0.6136
[10] Improvement: 1549.2686822158284	Time (s): 0.611
[11] Improvement: 1348.0894531891681	Time (s): 0.6166
[12] Improvement: 1387.5512039549649	Time (s): 0.6053
[13] Improvement: 1345.2979093464091	Time (s): 0.6085
[14] Improvement: 931.3408202352002	Time (s): 0.604
[15] Improvement: 645.6263559889048	Time (s): 0.6036
[16] Improvement: 510.3444970040582	Time (s): 0.6103
[17] Improvement: 434.5274738757871	Time (s): 0.611
[18] Improvement: 413.110292527359	Time (s): 0.6169
[19] Improvement: 353.6910662939772	Time (s): 0.611

[154] Improvement: 14.631833814550191	Time (s): 0.6102
[155] Improvement: 12.55102672567591	Time (s): 0.5925
[156] Improvement: 9.793359604198486	Time (s): 0.5895
[157] Improvement: 8.12817400507629	Time (s): 0.5992
[158] Improvement: 7.5996153363958	Time (s): 0.5937
[159] Improvement: 8.10697329044342	Time (s): 0.6147
[160] Improvement: 8.241170045454055	Time (s): 0.6148
[161] Improvement: 5.957488134503365	Time (s): 0.6161
[162] Improvement: 4.697395053692162	Time (s): 0.629
[163] Improvement: 4.037767232395709	Time (s): 0.6235
[164] Improvement: 3.4323450070805848	Time (s): 0.6185
[165] Improvement: 3.0401421836577356	Time (s): 0.6207
[166] Improvement: 2.978724332526326	Time (s): 0.6205
[167] Improvement: 3.2031098632141948	Time (s): 0.6256
[168] Improvement: 3.6936447671614587	Time (s): 0.5848
[169] Improvement: 4.35900957044214	Time (s): 0.5747
[170] Improvement: 4.967081101611257	Time (s): 0.5822
[171] Improvement: 5.553301138803363	Time (s): 0.605
[172] Improvement: 6.396350075

[306] Improvement: 3.451982125174254	Time (s): 0.6104
[307] Improvement: 6.833601897582412	Time (s): 0.6007
[308] Improvement: 11.343554568011314	Time (s): 0.6062
[309] Improvement: 8.808709786273539	Time (s): 0.6039
[310] Improvement: 12.231998468283564	Time (s): 0.6086
[311] Improvement: 12.458717925008386	Time (s): 0.6114
[312] Improvement: 10.318157609086484	Time (s): 0.6219
[313] Improvement: 8.716089860536158	Time (s): 0.6236
[314] Improvement: 7.437607718165964	Time (s): 0.6015
[315] Improvement: 6.215491747483611	Time (s): 0.62
[316] Improvement: 8.196780479513109	Time (s): 0.6245
[317] Improvement: 10.172288150060922	Time (s): 0.6058
[318] Improvement: 7.1169665227644145	Time (s): 0.6206
[319] Improvement: 7.832582802511752	Time (s): 0.6028
[320] Improvement: 9.782848776318133	Time (s): 0.6105
[321] Improvement: 7.053156598005444	Time (s): 0.608
[322] Improvement: 7.437384888995439	Time (s): 0.6033
[323] Improvement: 6.857943078037351	Time (s): 0.6129
[324] Improvement: 6.7819

[457] Improvement: 0.2977579836733639	Time (s): 0.6135
[458] Improvement: 0.3053094753995538	Time (s): 0.6139
[459] Improvement: 0.3201495078392327	Time (s): 0.625
[460] Improvement: 0.33855590550228953	Time (s): 0.6132
[461] Improvement: 0.35225527873262763	Time (s): 0.6351
[462] Improvement: 0.35283733205869794	Time (s): 0.6052
[463] Improvement: 0.34486137283965945	Time (s): 0.6223
[464] Improvement: 0.34406815096735954	Time (s): 0.635
[465] Improvement: 0.36308985110372305	Time (s): 0.6377
[466] Improvement: 0.4155167965218425	Time (s): 0.6314
[467] Improvement: 0.5393963726237416	Time (s): 0.6488
[468] Improvement: 0.830959958024323	Time (s): 0.6299
[469] Improvement: 1.4010936454869807	Time (s): 0.6117
[470] Improvement: 1.788879647385329	Time (s): 0.6115
[471] Improvement: 1.284793784841895	Time (s): 0.6201
[472] Improvement: 0.6915576998144388	Time (s): 0.6149
[473] Improvement: 0.5656315996311605	Time (s): 0.6243
[474] Improvement: 0.5999084766954184	Time (s): 0.6211
[475] Imp

[608] Improvement: 1.4298892230726779	Time (s): 0.6021
[609] Improvement: 0.6797389048151672	Time (s): 0.5708
[610] Improvement: 0.292207402177155	Time (s): 0.6099
[611] Improvement: 0.21474456880241632	Time (s): 0.6047
[612] Improvement: 0.2150702034123242	Time (s): 0.6058
[613] Improvement: 0.26027770433574915	Time (s): 0.6035
[614] Improvement: 0.3622351032681763	Time (s): 0.6168
[615] Improvement: 0.49149539275094867	Time (s): 0.6151
[616] Improvement: 0.49848091695457697	Time (s): 0.6075
[617] Improvement: 0.3656412633135915	Time (s): 0.6102
[618] Improvement: 0.2819134942255914	Time (s): 0.6125
[619] Improvement: 0.26101536862552166	Time (s): 0.6109
[620] Improvement: 0.2474712855182588	Time (s): 0.6123
[621] Improvement: 0.20849417662248015	Time (s): 0.6027
[622] Improvement: 0.15510936826467514	Time (s): 0.6077
[623] Improvement: 0.11634248215705156	Time (s): 0.6026
[624] Improvement: 0.0969014554284513	Time (s): 0.602
Total Improvement: 208773.48056401918
Total Time (s): 378.8

In [8]:
UNIQUE_PHONES = [
    'AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 
    'EH', 'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K', 
    'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 
    'TH', 'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH'
]

In [199]:
def make_phone_triple(bg_model):
    return tuple(bg_model.copy() for _ in range(3))

phone_distribution_triples = {
    phone: make_phone_triple(bg_model) for phone in UNIQUE_PHONES
}

In [214]:
def make_sentence_hmm(phones_cell, phone_triples, silence_model):
    phones = phones_cell.split(' ')
    cleaned_phones = wrap_with_silence(remove_stress_combinations(phones))
    hmm = pg.HiddenMarkovModel('-'.join(cleaned_phones))
    
    states = []
    for i, phone in enumerate(cleaned_phones):
        if phone == '_':
            phone_state = pg.State(silence_model.copy(), name='{}-SILENCE'.format(i))
            states.append(phone_state)
        else:
            phone_triple = phone_triples[phone]
            phone_states = (
                pg.State(phone_triple[0], name='{:02}a-{}'.format(i, phone)),
                pg.State(phone_triple[1], name='{:02}b-{}'.format(i, phone)),
                pg.State(phone_triple[2], name='{:02}c-{}'.format(i, phone))
            )
            states.extend(phone_states)
    hmm.add_states(states)
    
    hmm.add_transition(hmm.start, states[0], 1.0)
    for state in states:
        hmm.add_transition(state, state, 0.9)
    for prev, then in zip(states, states[1:]):
        hmm.add_transition(prev, then, 0.1)
    hmm.add_transition(states[-1], hmm.end, 0.1)
    hmm.bake()
    
    return hmm

example_sentence_hmm = make_sentence_hmm(
    script[script.sentence_id == 31].phones.iloc[0], 
    phone_distribution_triples, silence_model
)

In [219]:
unique_sentence_ids = enrollments_4_td.sentence_id.unique()

sentence_hmms = {
    sentence_id: make_sentence_hmm(
        script[script.sentence_id == sentence_id].phones.iloc[0], phone_distribution_triples, silence_model
    ) for sentence_id in unique_sentence_ids
    if isinstance(script[script.sentence_id == sentence_id].phones.iloc[0], str)
}

In [240]:
def update_triples(phone_triples):
    empty = 0
    lost = 0
    for phone_triple in phone_triples.values():
        for gmm in phone_triple:
            if gmm.summaries.sum() == 0:
                empty += 1
                continue
            try:
                gmm.from_summaries()
            except Exception:
                gmm.clear_summaries()
                lost += 1
    return empty, lost

def train_epoch(enrollments, unique_sentence_ids, sentence_hmms, phone_triples):
    logps = []
    
    for sentence_id in unique_sentence_ids:
        print(sentence_id, end=' ')
        if sentence_id not in sentence_hmms:
            continue
        sentence_hmm = sentence_hmms[sentence_id]
        
        subenrollments = enrollments[enrollments.sentence_id == sentence_id]
        sequences = [load_mfcc(path) for path in subenrollments['pcm_path']]
        weights = np.ones(len(sequences))
        
        logp = sentence_hmm.summarize(sequences=sequences, weights=weights, algorithm='viterbi', check_input=False)
        logps.append(logp)
    
    update_triples(phone_triples)
            
    return logps

In [245]:
for i in range(10):
    for j in range(10):
        logps = train_epoch(enrollments_4_td, unique_sentence_ids, sentence_hmms, phone_distribution_triples)
        print('{}-{} logps'.format(i, j), sum(logps))
    path = root + '/models/triples-{}'.format(i)
    os.mkdir(path)
    write_triples(path, phone_distribution_triples)

31 32 33 34 35 36 37 38 39 40 271 272 273 274 275 276 277 278 279 280 10087 10088 371 372 373 374 375 376 377 378 379 380 10107 10108 441 442 443 444 445 446 447 448 449 450 10121 10122 461 462 463 464 465 466 467 468 469 470 10125 10126 557 558 559 560 10143 10144 771 772 773 774 775 776 777 778 779 780 10187 10188 41 42 43 44 45 46 47 48 49 50 10041 10042 61 62 63 64 65 66 81 82 83 84 85 86 87 88 89 90 10049 10050 91 92 93 94 95 96 97 98 99 100 10051 10052 101 102 103 104 105 106 107 108 109 110 10053 10054 111 112 113 114 115 116 117 118 119 120 10055 10056 121 122 123 124 125 126 127 128 129 130 10057 10058 131 132 133 134 135 136 137 138 139 140 10059 10060 171 172 173 174 175 176 177 178 179 180 10067 10068 187 188 189 190 10069 10070 191 192 193 194 195 196 197 198 199 200 10071 10072 201 202 203 204 205 206 207 208 209 210 10073 10074 211 212 213 214 215 216 217 218 219 220 10075 10076 221 222 223 224 225 226 227 228 229 230 10077 10078 231 232 233 234 235 236 237 238 239 240 1

TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'

In [243]:
write_triples(root + '/models/triples', phone_distribution_triples)

In [244]:
got_this_back = read_triples(root + '/models/triples', UNIQUE_PHONES)

OSError: Cannot read model AW-b

In [105]:
print(example_sentence_hmm.states[3].distribution.distributions[0].mu)
print(example_sentence_hmm.states[10].distribution.distributions[0].mu)
print(example_sentence_hmm.states[10].name)

[  1.41467275e+01  -7.35344053e+00  -5.56301231e+00  -4.84370700e+00
  -3.26590152e+00  -4.67713110e+00  -2.93247692e+00  -6.21254481e+00
  -4.60166262e+00  -4.32767984e+00  -1.62883341e+00  -2.50476327e+00
  -8.24130769e+00   4.71618683e-01   4.57870227e-01  -1.32589895e+00
  -9.23610116e-01  -9.11397488e-01  -4.48633292e-02   6.83980317e-01
  -3.91819699e-01  -9.57840195e-02  -5.51740595e-01  -4.66018632e-01
  -3.27380013e-01   2.68423365e-03   6.40123001e-03   1.34323619e-01
   6.15838646e-03  -4.11852850e-02   7.89280213e-02  -2.26739330e-02
   5.75374672e-02   6.40953838e-03  -3.25918412e-02   1.42842274e-01
  -1.47236299e-01  -6.17011349e-02  -1.74521876e-02]
[  1.43344790e+01   1.12123728e-01  -4.61673784e+00  -4.37537285e+00
  -3.35688364e+00  -5.19159258e+00  -2.46247097e+00  -5.27950688e+00
  -6.93267329e+00   8.92074965e-01  -1.50063469e+00  -3.92398961e+00
  -1.10409868e+01   2.92374934e-01   1.52636161e+00  -1.38141933e+00
  -6.50400994e-01  -7.84072233e-01   1.30328483e-0

In [71]:
for i in (0, 50, 100):
    predictions = example_sentence_hmm.predict(example_features[i], algorithm='viterbi')
    print(i)
    print('\t'.join(example_sentence_hmm.states[prediction].name for prediction in predictions))

0
_-M-AY-_-V-OY-S-_-IH-Z-_-M-AY-_-P-AE-S-W-ER-D-_-start	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	1-M-beg	1-M-mid	1-M-end	2-AY-beg	2-AY-mid	2-AY-end	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-S

In [76]:
example_sentence_hmm.fit(example_features, algorithm='viterbi', n_jobs=12, verbose=True)

TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._from_summaries'
Traceback (most recent call last):
  File "pomegranate/bayes.pyx", line 710, in pomegranate.bayes.BayesModel.from_summaries
  File "pomegranate/distributions.pyx", line 2259, in pomegranate.distributions.MultivariateGaussianDistribution.from_summaries
TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'


TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._from_summaries'
Traceback (most recent call last):
  File "pomegranate/bayes.pyx", line 710, in pomegranate.bayes.BayesModel.from_summaries
  File "pomegranate/distributions.pyx", line 2259, in pomegranate.distributions.MultivariateGaussianDistribution.from_summaries
TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'


[1] Improvement: 4.373646683990955	Time (s): 522.3


...

[61] Improvement: 2.1627172827720642e-05	Time (s): 537.9


TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._from_summaries'
Traceback (most recent call last):
  File "pomegranate/bayes.pyx", line 710, in pomegranate.bayes.BayesModel.from_summaries
  File "pomegranate/distributions.pyx", line 2259, in pomegranate.distributions.MultivariateGaussianDistribution.from_summaries
TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'


[62] Improvement: -1.4025717973709106e-06	Time (s): 544.4
Total Training Improvement: 2551.5645321204793
Total Training Time (s): 33826.2032


2551.5645321204793

## Enroll

In [54]:
def make_speaker_classifier(inspeaker_classifier):
    n_components = 3 * len(phones)
    initial, transitions = make_matrix_beads(n_components)

    classifier = hmm.GMMHMM(
        n_components=n_components, n_mix=8, covariance_type='diag', init_params='mcw', params='tmcw'
    )

    classifier.startprob_ = initial
    classifier.transmat_ = transitions

GMMHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
    init_params='stmcw', n_components=39, n_iter=10, n_mix=8,
    params='stmcw', random_state=None, startprob_prior=1.0, tol=0.01,
    transmat_prior=1.0, verbose=False)

In [ ]:
def perform_enrollments(classifier, enrollments):
    labels = enrollments[['is_male', 'speaker_id', 'sentence_id']].values
    features = [load_mfcc(path) for path in enrollments_1['pcm_path']]
    classifier.fit(features, labels)
    

## Trial

In [ ]:
def save_results(label, results):
    path = os.path.join(root, 'result', label)
    with open(path) as opened:
        pickle.dump(results, opened)
        
def load_results(label):
    path = os.path.join(root, 'result', label)
    with open(path) as opened:
        return pickle.load(opened)
    
def perform_trial(classifier, path):
    features = load_mfcc(path)
    return classifier.predict_single_proba(features)

def perform_trials(classifier, trials):
    paths = trials['pcm_path'].unique()
    results = {}
    for path in paths:
        results[path] = perform_trial(classifier, path)
    return results

In [10]:
per_phone_weights = {}

per_phone_weights.get('A', np.eye(3))

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [ ]:
class Bakis:
    def __init__(self, phones, per_phone_weights, ignore_stress=True, wrap_silence=True):
        self.phone_seq = phones.split(' ')
        if wrap_silence:
            self.phone_seq = ['_'] + self.phone_seq + ['_']
            
        state_transitions = 

In [51]:
enrollments_1.sentence_id.unique()

array([31, 32, 33, 34, 35, 36, 37, 38, 39, 40])

In [52]:
sample_enrollments = enrollments_1.loc[enrollments_1.sentence_id == 31]

In [ ]:
class HmmGmmVerifier:
    def __init__(self):
        self.patterns = None
        self.labels = None

    def fit(self, features, labels):
        self.patterns = features
        self.labels = labels
        self.unique_labels = np.unique(labels)
        
    def enroll(self, ):
        pass
    
    def trial(self, ):
        pass

    def predict(self, features, metric=spatial.distance.cosine):
        sequence_label_proba = self.predict_proba(features, metric)
        max_proba_index = sequence_label_proba.argmax(axis=1)
        return self.unique_labels[max_proba_index]
    
    def predict_proba(self, features, metric=spatial.distance.cosine):
        sequence_n = len(features) 
        pattern_n = len(self.patterns)
        
        sequence_label_proba = np.zeros((sequence_n, pattern_n), dtype=self.labels.dtype)
        for i, sequence in enumerate(features):
            sequence_label_proba[i, :] = self.predict_single_proba(sequence, metric)
            
        return sequence_label_proba
    
    def predict_single_proba(self, sequence, metric=spatial.distance.cosine):
        pattern_dists = np.zeros(len(self.patterns), dtype=np.float64)
        for i, pattern in enumerate(self.patterns):
            distance, _ = fastdtw.fastdtw(pattern, sequence, dist=metric)
            pattern_dists[i] = distance
            
        pattern_proba = np.exp(-pattern_dists)
        
        label_proba = np.zeros(len(self.unique_labels), dtype=np.float64)
        all_dim = tuple(range(1, self.labels.ndim))
        for i, label in enumerate(self.unique_labels):
            relevant = (self.labels == label).all(axis=all_dim)
            total_proba = pattern_proba[relevant].sum()
            label_proba[i] = total_proba
        
        return label_proba / label_proba.sum()

classifier = markov.HMMGMMClassifier()
classifier.fit(e_features, e_labels, n_components=10, n_mix=2)
return classifier.predict(t_features)

In [5]:
display(enrollments_1.groupby(['is_male', 'speaker_id']).size())

is_male  speaker_id
False    2             30
         4             30
         5             30
         6             30
         8             24
         12            30
True     1             30
         2             30
         4             30
         5             30
         6             30
         7             30
         8              6
         9             30
         13            30
         14            24
         15            30
         16            30
         17            30
         18            30
         19            30
         20            30
         21            24
         22            30
         23            30
         26            30
         28            30
         29            30
         32            30
         38            24
         40            30
         41            30
         43            30
         47            30
         48             6
         51            30
         52             6
         53       